In [6]:
import datetime

import numpy as np
import pandas as pd
import yaml

In [27]:
from sqlalchemy import create_engine

with open('./config.yml', 'r') as f:
        config = yaml.safe_load(f)
        config_ryf = config['RAPIDO_Y_FURIOSO']
        config_etl = config['ETL']

url_ryf = (f"{config_ryf['drivername']}://{config_ryf['user']}:{config_ryf['password']}@{config_ryf['host']}:"
        f"{config_ryf['port']}/{config_ryf['dbname']}")
url_etl = (f"{config_etl['drivername']}://{config_etl['user']}:{config_etl['password']}@{config_etl['host']}:"
        f"{config_etl['port']}/{config_etl['dbname']}")

ryf = create_engine(url_ryf)
etl_conn = create_engine(url_etl)

In [45]:
df_trans = pd.read_sql_table('trans_novedad', etl_conn)
dim_ciudad = pd.read_sql_table('dim_ciudad', etl_conn)
dim_cliente = pd.read_sql_table('dim_cliente', etl_conn)
dim_estado = pd.read_sql_table('dim_estado', etl_conn)
dim_fecha = pd.read_sql_table('dim_fecha', etl_conn)
dim_mensajero = pd.read_sql_table('dim_mensajero', etl_conn)
dim_sede = pd.read_sql_table('dim_sede', etl_conn)

In [46]:
fact_servicio = pd.read_sql_table('mensajeria_servicio', ryf)

In [47]:
fact_servicio['fecha_solicitud'] = pd.to_datetime(fact_servicio['fecha_solicitud'])
dim_fecha['fecha'] = pd.to_datetime(dim_fecha['fecha'])

fact_servicio = fact_servicio.merge(dim_fecha, left_on='fecha_solicitud', right_on='fecha', how='left')
fact_servicio = fact_servicio[['fecha_solicitud', 'hora_solicitud', 'key_dim_fecha', 'dia_semana', 'mes_str', 'es_festivo', 'fecha_str', 'cliente_id', 'mensajero_id', 'ciudad_origen_id', 'ciudad_destino_id']]

fact_servicio.head()

,fecha_solicitud,hora_solicitud,key_dim_fecha,dia_semana,mes_str,es_festivo,fecha_str,cliente_id,mensajero_id,ciudad_origen_id,ciudad_destino_id
0,2023-10-26,09:46:03,290,3,October,False,26/10/2023,5,NaN,1,1
1,2023-10-26,11:18:14,290,3,October,False,26/10/2023,5,7.0,1,1
2,2023-10-28,19:21:01,292,5,October,False,28/10/2023,5,NaN,1,1
3,2023-11-07,09:46:09,302,1,November,False,07/11/2023,5,NaN,1,1
4,2023-11-07,09:46:10,302,1,November,False,07/11/2023,5,NaN,1,1


In [49]:
fact_servicio.replace({
    np.nan: 'no aplica',
    'NaT': 'no aplica'
}, inplace=True)

fact_servicio.head()

,fecha_solicitud,hora_solicitud,key_dim_fecha,dia_semana,mes_str,es_festivo,fecha_str,cliente_id,mensajero_id,ciudad_origen_id,ciudad_destino_id
0,2023-10-26,09:46:03,290,3,October,False,26/10/2023,5,no aplica,1,1
1,2023-10-26,11:18:14,290,3,October,False,26/10/2023,5,7.0,1,1
2,2023-10-28,19:21:01,292,5,October,False,28/10/2023,5,no aplica,1,1
3,2023-11-07,09:46:09,302,1,November,False,07/11/2023,5,no aplica,1,1
4,2023-11-07,09:46:10,302,1,November,False,07/11/2023,5,no aplica,1,1


In [ ]:
fact_servicio = fact_servicio.merge(dim_cliente, left_on='cliente_id', right_on='Cliente_Key', how='left')
fact_servicio['Cliente_Key'] = fact_servicio['Cliente_Key']